# 301: Itinerary Choice using MNL

In [ ]:
import larch.numba as lx

This example is an itinerary choice model built using the example
itinerary choice dataset included with Larch.  We'll begin by loading
that example data.

In [ ]:
d = lx.Dataset.from_idce(
    pd.read_csv(lx.example_file("arc"), index_col=['id_case','id_alt']),
)

In [ ]:
# TEST
assert d['choice'].min() == 0

Now let's make our model.  We'll use a few variables to define our
linear-in-parameters utility function.

In [ ]:
m = lx.Model(datatree=d)

v = [
    "timeperiod==2",
    "timeperiod==3",
    "timeperiod==4",
    "timeperiod==5",
    "timeperiod==6",
    "timeperiod==7",
    "timeperiod==8",
    "timeperiod==9",
    "carrier==2",
    "carrier==3",
    "carrier==4",
    "carrier==5",
    "equipment==2",
    "fare_hy",    
    "fare_ly",    
    "elapsed_time",  
    "nb_cnxs",       
]


The `larch.roles` module defines a few convenient classes for declaring data and parameter.
One we will use here is `PX` which creates a linear-in-parameter term that represents one data
element (a column from our data, or an expression that can be evaluated on the data alone) multiplied
by a parameter with the same name.


In [ ]:
from larch.roles import PX
m.utility_ca = sum(PX(i) for i in v)
m.choice_ca_var = 'choice'
m.availability_var = 1

In [ ]:
# TEST
from pytest import approx
assert m.loglike() == approx(-953940.4411159059)

Since we are estimating just an MNL model in this example, this is all we need to do to build
our model, and we're ready to go.  To estimate the likelihood maximizing parameters, we give:


In [ ]:
m.maximize_loglike()

In [ ]:
# TEST
r = _
assert r.loglike == approx(-777770.0688722524)
assert r.method == 'bhhh'
assert r.x.to_dict() == approx({
    'carrier==2': 0.117200479172321,
    'carrier==3': 0.638554248463955,
    'carrier==4': 0.5652518834386212,
    'carrier==5': -0.6240216384597791,
    'elapsed_time': -0.006086926416874131,
    'equipment==2': 0.46630452724649335,
    'fare_hy': -0.0011750130617308708,
    'fare_ly': -0.0011772411112551142,
    'nb_cnxs': -2.9471531121829315,
    'timeperiod==2': 0.09594946765414832,
    'timeperiod==3': 0.12653314846972896,
    'timeperiod==4': 0.06055186075075148,
    'timeperiod==5': 0.14096321358526923,
    'timeperiod==6': 0.23825430881925683,
    'timeperiod==7': 0.3513907859210803,
    'timeperiod==8': 0.3533021391793547,
    'timeperiod==9': -0.010309386019381033,
})